In [1]:
# Full notebook for calculating Vaccination-dependent IFR and cases as a function of IFR
# Vaccination rate: V(t) 
# IFR(t) -> IFR(V(t)) 
# Deaths(t) -> Deaths(IFR(V(t))

import numpy as np
import pandas as pd
pd.set_option('display.max_rows', 500)
pd.set_option('display.min_rows', 50)

import matplotlib.pyplot as plt
%matplotlib widget
plt.rcParams['figure.figsize'] = (12,8)
plt.rcParams["image.cmap"] = "tab10"
plt.rcParams['axes.prop_cycle'] = plt.cycler(color=plt.cm.tab10.colors)
fs_label = 16
parameters = {
                'axes.labelsize': fs_label,
                'axes.titlesize': fs_label+4,
                'xtick.labelsize': fs_label,
                'ytick.labelsize': fs_label, 
                'lines.markersize': 10,
                'lines.linewidth': 3
             }
plt.rcParams.update(parameters)
%matplotlib widget
import matplotlib.colors as colors
from matplotlib import cm # Colormaps

cmap = cm.get_cmap('Dark2',7)

import locale
import matplotlib.dates as mdates
# locale.setlocale(locale.LC_TIME,"Danish")

# ax1 = plt.gca()
# ax1.xaxis.set_major_formatter(mdates.DateFormatter('%b\n%Y'))


import os
import math

from datetime import date
import datetime

saveFigures = True
print('saveFigures is set to: '+str(saveFigures))
print('Done loading packages')

def rnMean(data,meanWidth):
    return np.convolve(data, np.ones(meanWidth)/meanWidth, mode='valid')
def rnTime(t,meanWidth):
    return t[math.floor(meanWidth/2):-math.ceil(meanWidth/2)+1]
def rnTime2(t,meanWidth):
    return t[meanWidth-1:]
def plotWithMean(ax,x,y,meanWidth):
    firstLine = ax.plot(x,y,'.:',markersize=2,linewidth=0.5)
    firstColor = firstLine[0].get_color()
    ax.plot(rnTime(x,meanWidth),rnMean(y,meanWidth),color=firstColor)

saveFigures is set to: True
Done loading packages


# Get vaccinations as a function of time

In [2]:
dataPath = './data/OWID_vaccinations.csv'

dfVacc = pd.read_csv(dataPath)

dfVacc.date = pd.to_datetime(dfVacc.date)
dkdf = dfVacc[dfVacc.location == 'Denmark']

In [3]:
# plt.figure()
# plt.plot(dfVacc.date,dfVacc.people_vaccinated_per_hundred,'*k')
# plt.show()

In [4]:
# Make a dataframe for vaccination-level in all places at all possible days
# firstPosVaccDate = dfVacc.loc[dfVacc.people_vaccinated_per_hundred.isna() == False].date.min()
# lastPosVaccDate = dfVacc.loc[dfVacc.people_vaccinated_per_hundred.isna() == False].date.max()
firstPosVaccDate = dfVacc.date.min()
lastPosVaccDate = dfVacc.date.max()

allVaccDates = np.arange(firstPosVaccDate,lastPosVaccDate,np.timedelta64(1,'D'))


dfVaccOnce = pd.DataFrame()
dfVaccOnce['Dates'] = allVaccDates

for curReg in dfVacc.location.unique():
    
    # Get current region
    thisdf = dfVacc[dfVacc.location == curReg]

    # Make a dataframe of just dates and people_vaccinated_per_hundred
    testdf = pd.DataFrame()
    testdf['Dates'] = thisdf.date
    testdf[curReg] = thisdf.people_vaccinated_per_hundred
    # ... and merge it with the full dataframe
    dfVaccOnce = dfVaccOnce.merge(testdf,how='left')

In [5]:
# Set index 
dfVaccOnce.set_index('Dates',inplace=True)

# Set all countries to 0 on the first day
zeroRow = np.zeros(len(dfVaccOnce.iloc[0]))
dfVaccOnce.iloc[0] = zeroRow

# Interpolate all values
for col in dfVaccOnce.columns:
    dfVaccOnce[col] = dfVaccOnce[col].interpolate().values.ravel().tolist()

# Reset index
dfVaccOnce.reset_index(inplace=True)

## Alternative: Set 0 on first day, and on the day before first real data-point, to avoid non-zero until there's actually data

In [6]:
# dfVaccOnce['Peru'].unique()

# # print(thisdf.loc[40058])
# # print(thisdf.loc[40059])
# # print(thisdf.loc[40060])

# # nanFilled = linearly_interpolate_nans(thisdf.people_vaccinated_per_hundred.values)
# nanFilled = thisdf.people_vaccinated_per_hundred.interpolate().values.ravel().tolist()

# plt.figure()
# plt.plot(thisdf.date,thisdf.people_vaccinated_per_hundred,'*-')
# plt.plot(thisdf.date,nanFilled,'.k')
# dfVaccOnce[dfVaccOnce.columns[30:50]].plot()

# Calculate vaccination-dependent IFR's for all (possible) countries, using Marc Bevands method

In [7]:
# Pyramid data is from the United Nations: this file is a CSV export of the first sheet
# of "Population by Age Groups - Both Sexes" linked from:
# https://population.un.org/wpp/Download/Standard/Population/
# Direct link:
# https://population.un.org/wpp/Download/Files/1_Indicators%20(Standard)/EXCEL_FILES/1_Population/WPP2019_POP_F07_1_POPULATION_BY_AGE_BOTH_SEXES.xlsx
file_pyramids = 'data/WPP2019_POP_F07_1_POPULATION_BY_AGE_BOTH_SEXES.csv'

maxage = 100
# Age groups defined in the CSV file
age_groups = [(0,4), (5,9), (10,14), (15,19), (20,24), (25,29), (30,34), (35,39), (40,44), (45,49), (50,54), (55,59), (60,64), (65,69), (70,74), (75,79), (80,84), (85,89), (90,94), (95,99), (100,maxage)]

# This will hold parsed pyramid data. Example to get the number of people in the
# age group 20-24 in France: pyramid['France'][(20,24)]
pyramid = {}

def ag2str(age_group):
    if age_group[1] == maxage:
        return f'{age_group[0]}+'
    return f'{age_group[0]}-{age_group[1]}'

def parse_pyramids():
    df = pd.read_csv(file_pyramids)
    # ignore labels as they don't contain any data
    df = df[df['Type'] != 'Label/Separator']
    # only take rows with data as of 2020
    df = df[df['Reference date (as of 1 July)'] == 2020]
    # only parse countries, world, and continents
    df = df[df['Type'].isin(('Country/Area', 'World', 'Region'))]
    # remove spaces used as thousands separators, and convert cell values to floats
    columns = [ag2str(x) for x in age_groups]
    for col in columns:
        df[col] = df[col].str.replace('\s+', '').astype(float)
    regions = list(df['Region, subregion, country or area *'])
    #regions = ('France',)
    for region in regions:
        pyramid[region] = {}
        df_region = df[df['Region, subregion, country or area *'] == region]
        for ag in age_groups:
            # values are in thousands
            pyramid[region][ag] = 1000 * float(df_region[ag2str(ag)])

# Run parse_pyramids
parse_pyramids()


def people_of_age(pyramid_region, age):
    # Returns the number of people of exact age 'age', given the provided age pyramid
    for ((a, b), n) in pyramid_region.items():
        if age in range(a, b + 1):
            return n / float(b - a + 1)


<ipython-input-7-1c2a2470a5b4>:32: FutureWarning: The default value of regex will change from True to False in a future version.
  df[col] = df[col].str.replace('\s+', '').astype(float)


In [8]:
# Age-stratified IFR estimates for COVID-19, as compiled by Marc Bevand
# Some sources have been omitted due to problems with missing agegroups.
# With some additional work, these could maybe be included
ifrs_covid = [

        # Calculated from Spanish ENE-COVID study
        # (see calc_ifr.py)
        ('ENE-COVID', {
            (0,9):    0.003,
            (10,19):  0.004,
            (20,29):  0.015,
            (30,39):  0.030,
            (40,49):  0.064,
            (50,59):  0.213,
            (60,69):  0.718,
            (70,79):  2.384,
            (80,89):  8.466,
            (90,maxage): 12.497,
        }),

        # US CDC estimate as of 19 Mar 2021
        # https://www.cdc.gov/coronavirus/2019-ncov/hcp/planning-scenarios.html
        # (table 1)
        ('US CDC', {
            (0,17):   0.002,
            (18,49):  0.05,
            (50,64):  0.6,
            (65,maxage): 9.0,
        }),

        # Verity et al.
        # https://www.thelancet.com/journals/laninf/article/PIIS1473-3099(20)30243-7/fulltext
        # (table 1)
        ('Verity', {
            (0,9):    0.00161,
            (10,19):  0.00695,
            (20,29):  0.0309,
            (30,39):  0.0844,
            (40,49):  0.161,
            (50,59):  0.595,
            (60,69):  1.93,
            (70,79):  4.28,
            (80,maxage): 7.80,
        }),

        # Levin et al.
        # https://link.springer.com/article/10.1007/s10654-020-00698-1
        # (table 3)
        ('Levin', {
            (0,34):   0.004,
            (35,44):  0.068,
            (45,54):  0.23,
            (55,64):  0.75,
            (65,74):  2.5,
            (75,84):  8.5,
            (85,maxage): 28.3,
        }),

        # Salje et al.: Estimating the burden of SARS-CoV-2 in France
        # https://science.sciencemag.org/content/369/6500/208
        # Supplementary Materials:
        # https://science.sciencemag.org/content/sci/suppl/2020/05/12/science.abc3517.DC1/abc3517_Salje_SM_rev2.pdf
        # (table S2)
        ('Salje', {
            (0,19):   0.001,
            (20,29):  0.005,
            (30,39):  0.02,
            (40,49):  0.05,
            (50,59):  0.2,
            (60,69):  0.7,
            (70,79):  1.9,
            (80,maxage):  8.3,
        }),

        # # Perez-Saez et al.
        # # https://www.thelancet.com/journals/laninf/article/PIIS1473-3099(20)30584-3/fulltext
        # ('Perez-Saez', {
        #     (5,9):    0.0016,
        #     (10,19):  0.00032,
        #     (20,49):  0.0092,
        #     (50,64):  0.14,
        #     (65,maxage): 5.6,
        # }),

        # # Picon et al.
        # # https://www.ncbi.nlm.nih.gov/pmc/articles/PMC7493765/
        # # (table 2)
        # ('Picon', {
        #     (20,39):  0.08,
        #     (40,59):  0.24,
        #     (60,maxage):  4.63,
        # }),

        # Poletti et al.
        # https://www.eurosurveillance.org/content/10.2807/1560-7917.ES.2020.25.31.2001383
        # (table 1, column "Any time")
        ('Poletti', {
            (0,19):   0,
            (20,49):  0,
            (50,59):  0.46,
            (60,69):  1.42,
            (70,79):  6.87,
            (80,maxage):  18.35,
        }),

        # # Gudbjartsson et al.: Humoral Immune Response to SARS-CoV-2 in Iceland
        # # https://www.nejm.org/doi/full/10.1056/NEJMoa2026116
        # # Supplementary Appendix 1
        # # https://www.nejm.org/doi/suppl/10.1056/NEJMoa2026116/suppl_file/nejmoa2026116_appendix_1.pdf
        # # (table S7)
        # ('Gudbjartsson', {
        #     (0,70):   0.1,
        #     (71,80):  2.4,
        #     (81,maxage): 11.2,
        # }),

        # Public Health Agency of Sweden
        # https://www.folkhalsomyndigheten.se/contentassets/53c0dc391be54f5d959ead9131edb771/infection-fatality-rate-covid-19-stockholm-technical-report.pdf
        # (table B.1)
        ('PHAS', {
            (0,49):   0.01,
            (50,59):  0.27,
            (60,69):  0.45,
            (70,79):  1.92,
            (80,89):  7.20,
            (90,maxage):  16.21,
        }),

        # O’Driscoll et al.: Age-specific mortality and immunity patterns of SARS-CoV-2
        # https://www.nature.com/articles/s41586-020-2918-0
        # Supplementary information
        # https://static-content.springer.com/esm/art%3A10.1038%2Fs41586-020-2918-0/MediaObjects/41586_2020_2918_MOESM1_ESM.pdf
        # (table S3)
        ('O’Driscoll', {
            (0,4):   0.003,
            (5,9):   0.001,
            (10,14): 0.001,
            (15,19): 0.003,
            (20,24): 0.006,
            (25,29): 0.013,
            (30,34): 0.024,
            (35,39): 0.040,
            (40,44): 0.075,
            (45,49): 0.121,
            (50,54): 0.207,
            (55,59): 0.323,
            (60,64): 0.456,
            (65,69): 1.075,
            (70,74): 1.674,
            (75,79): 3.203,
            (80,maxage): 8.292,
        }),

        # # Ward et al.: Antibody prevalence for SARS-CoV-2 in England following first peak of the pandemic: REACT2 study in 100,000 adults
        # # https://www.nature.com/articles/s41467-021-21237-w
        # # (table 2)
        # ('REACT2', {
        #     (15,44):  0.03,
        #     (45,64):  0.52,
        #     (65,74):  3.13,
        #     (75,maxage): 11.64,
        # }),

        # Yang et al.: Estimating the infection fatality risk of COVID-19 in New York City during the spring 2020 pandemic wave
        # https://www.medrxiv.org/content/10.1101/2020.06.27.20141689v2
        # (table 1)
        ('Yang', {
            (0,24):   0.0097,
            (25,44):  0.12,
            (45,64):  0.94,
            (65,74):  4.87,
            (75,maxage): 14.17,
        }),

        # Molenberghs et al.: Belgian Covid-19 Mortality, Excess Deaths, Number of Deaths per Million, and Infection Fatality Rates
        # https://www.medrxiv.org/content/10.1101/2020.06.20.20136234v1
        # (table 6)
        ('Molenberghs', {
            (0,24):   0.0005,
            (25,44):  0.017,
            (45,64):  0.21,
            (65,74):  2.24,
            (75,84):  4.29,
            (85,maxage): 11.77,
        }),

        # Brazeau et al.
        # https://www.imperial.ac.uk/mrc-global-infectious-disease-analysis/covid-19/report-34-ifr/
        # (table 2, column "IFR (%) with Seroreversion")
        ('Brazeau', {
            (0,4):    0.00,
            (5,9):    0.01,
            (10,14):  0.01,
            (15,19):  0.02,
            (20,24):  0.02,
            (25,29):  0.04,
            (30,34):  0.06,
            (35,39):  0.09,
            (40,44):  0.15,
            (45,49):  0.23,
            (50,54):  0.36,
            (55,59):  0.57,
            (60,64):  0.89,
            (65,69):  1.39,
            (70,74):  2.17,
            (75,79):  3.39,
            (80,84):  5.30,
            (85,89):  8.28,
            (90,maxage): 16.19,
        }),

]

In [9]:
def overall_ifr(pyramid_region, ifr_age_stratified):
    pop = 0
    deaths = 0
    for (age_group, ifr) in ifr_age_stratified.items():
        for age in range(age_group[0], age_group[1] + 1):
            pop += people_of_age(pyramid_region, age)
            deaths += people_of_age(pyramid_region, age) * ifr / 100.0
            
    return 100.0 * deaths / pop

In [10]:
# Test
print('Test of functions:')
curPyr = pyramid['Denmark']
print(f'DK, ENE-covid: {overall_ifr(pyramid_region=curPyr,ifr_age_stratified=ifrs_covid[0][1])}')
print(f'DK, US-CDC: {overall_ifr(pyramid_region=curPyr,ifr_age_stratified=ifrs_covid[1][1])}')

Test of functions:
DK, ENE-covid: 0.7970310827145569
DK, US-CDC: 1.9538892764634779


In [11]:
# Make a function which calculates all possible ifrs and takes the mean
def effective_ifr_mean(curPyramid):
    allIFRs = []
    for curIFRCount in range(len(ifrs_covid)):
        # print(curIFRCount)
        allIFRs.append(overall_ifr(pyramid_region=curPyramid,ifr_age_stratified=ifrs_covid[curIFRCount][1]))

    return np.mean(allIFRs)
        
# And test
curPyr = pyramid['Denmark']
print(f'DK, mean IFR: {effective_ifr_mean(curPyr)}')


DK, mean IFR: 1.2025797563617526


In [12]:
# Function for getting pyramid at a given vaccination-rate

def getVaccPyr(defPyramid,ratioVacc): 
    # Get the full size of the population
    fullPop = np.fromiter(defPyramid.values(), dtype=float).sum()
    # Get possible ages
    keys = list(defPyramid.keys())

    # Copy the pyramid   
    curPyr = defPyramid.copy()

    # Figure out how many to remove
    toRemoveTot = np.ceil(fullPop * (ratioVacc))

    # Set counter to keep track of how many left to remove
    toRemove = toRemoveTot

    # Go through pyramid, starting at oldest
    for k in range(len(curPyr)-1,-1,-1):
        # Get the current age and count
        thisKey = keys[k]
        thisCount = curPyr[thisKey]
        
        # If the number to remove exceeds the size of the population, remove all from pyramid and reduce toRemove
        if (toRemove >= thisCount):
            curPyr[thisKey] = 0
            toRemove -= thisCount

        # Otherwise, remove the remaining and set toRemove to zero
        else:
            curPyr[thisKey] -= toRemove
            # curPyr[thisKey] = np.floor(curPyr[thisKey] - toRemove)
            toRemove = 0
            # Set k to -1 to end loop
            k = -1 
    return curPyr

In [13]:
# Test
defPyr = pyramid['Denmark']
getVaccPyr(defPyr,0.85)

{(0, 4): 309000.0,
 (5, 9): 297000.0,
 (10, 14): 262650.0,
 (15, 19): 0,
 (20, 24): 0,
 (25, 29): 0,
 (30, 34): 0,
 (35, 39): 0,
 (40, 44): 0,
 (45, 49): 0,
 (50, 54): 0,
 (55, 59): 0,
 (60, 64): 0,
 (65, 69): 0,
 (70, 74): 0,
 (75, 79): 0,
 (80, 84): 0,
 (85, 89): 0,
 (90, 94): 0,
 (95, 99): 0,
 (100, 100): 0}

# Make dataframe of time-dependent effective IFR's using functions defined above

In [14]:
dfIFR = pd.DataFrame()
dfIFR['Dates'] = dfVaccOnce['Dates']

regionsNotFound = []
for curReg in dfVaccOnce.columns[1:]:
    # Get the current pyramid    
    try: 
        curDefPyramid = pyramid[curReg]
    except:
        regionsNotFound.append(curReg)
        # print(f'Region {curReg} not found')
        continue
    # print(curReg)

    # Translate a given vaccination-rate to an IFR, using the mean of all IFRs
    dfIFR[curReg] = dfVaccOnce[curReg].apply(lambda x: effective_ifr_mean(getVaccPyr(curDefPyramid,x/100)))
    

    # # Translate a given vaccination-rate to an IFR
    # # Apply-version
    # # dfIFR[curReg] = dfVaccOnce[curReg].apply(lambda x: overall_ifr(pyramid_region=getVaccPyr(curDefPyramid,x/100),ifr_age_stratified=ifrs_covid[0][1]))
    # dfIFR[curReg] = dfVaccOnce[curReg].apply(lambda x: overall_ifr(pyramid_region=getVaccPyr(curDefPyramid,x/100),ifr_age_stratified=ifrs_covid[2][1])) # Verity

    # # # Loop-version (appears to be same speed)
    # # for vaccVal in dfVaccOnce[curReg].values:
    # #     # Calculate one IFR
    # #     curIFR = overall_ifr(pyramid_region=getVaccPyr(curDefPyramid,vaccVal/100),ifr_age_stratified=ifrs_covid[0][1])


# --- NB ---
# Loop takes about 224 seconds to run on Rasmus' desktop.
# If speed is important, use a single IFR below, as the Verity IFR is relatively close to the total average.
# This should reduce the time by a factor 11 

In [ ]:
# # dfVaccOnce[curReg]
# # curDefPyramid
# # dfVaccOnce[curReg].values
# # overall_ifr(pyramid_region=getVaccPyr(curDefPyramid,1),ifr_age_stratified=ifrs_covid[0][1])
# # overall_ifr(pyramid_region=getVaccPyr(defPyr,0.85),ifr_age_stratified=ifrs_covid[0][1])
# # len(ifrs_covid) 

# dfIFR['asdf'] = dfVaccOnce['Denmark'].apply(lambda x: overall_ifr(pyramid_region=getVaccPyr(curDefPyramid,x/100),ifr_age_stratified=ifrs_covid[0][1]))

# dfIFR['Denmark']

# Load data and calculate cases from deaths

In [ ]:
# Load data
dataPath = './data/OWID_data.csv'

dfData = pd.read_csv(dataPath)

# print(dfData.columns)

dfData.date = pd.to_datetime(dfData.date)

curdfData = dfData[dfData.location == 'Denmark']
# curCase = deathsToUse/(np.divide(ifrpertime,100))
# curCaseMean = deathsToUse/(ifrpertime[0]/100)


# Plot results

In [ ]:
# dataDates = curdfData.date.values.astype('datetime64[D]')
# curDates = dfIFR.Dates.values.astype('datetime64[D]')
# missingDates = np.arange(dataDates[0],curDates[0],np.timedelta64(1,'D'))

# allDates = np.append(missingDates,curDates)

# regVacc = dfVaccOnce[curRegion]
# regIFR = dfIFR[curRegion].values

# allVacc = np.append(np.zeros((len(missingDates),)),regVacc)
# allIFR = np.append(regIFR[0]*np.ones((len(missingDates),)),regIFR)

# # ifrpertime = []

# # for k in range(0,len(allDkVacc)):
# #     curVacc = allDkVacc[k]
# #     curIndex = np.where(np.round((ifrVaccDF['VaccRate']*100)).astype(int) == curVacc)[0]
# #     curIFR = ifrVaccDF['Denmark'].iloc[curIndex].values[0]
# #     ifrpertime.append(curIFR)

# len(allDates)
# len(allIFR)
# len(deathsToUse/allIFR)

# curCase = deathsToUse/(np.divide(allIFR,100))
# curCaseMean = deathsToUse/(allIFR[0]/100)
# len(allIFR)
# len(deathsToUse)

In [ ]:
# # plt.figure()
# # plt.plot(curdfData.date,curdfData.new_deaths)
# len(missingDates)
# curRegion = 'Denmark'
# # curRegion = 'Sweden'
# curRegion = 'Germany'
# dfData[dfData.location == curRegion].date.values[0]

# plt.figure()
# plt.plot(deathsToUse)
# regionsToPlot = dfIFR.columns[1:]
# regionsToPlot

In [ ]:
regionsToPlot = ['Denmark','Germany','Sweden']
# regionsToPlot = dfIFR.columns[1:]
curRegion = regionsToPlot[0]

for curRegion in regionsToPlot:

    try:

        fig,(ax1,ax2) = plt.subplots(2,1,sharex=True,figsize=(14,14),gridspec_kw={'height_ratios': [3, 1]})

        meanWidth = 14

        # Get data
        curdfData = dfData[dfData.location == curRegion]

        dataDates = curdfData.date.values.astype('datetime64[D]')
        curDates = dfIFR.Dates.values.astype('datetime64[D]')
        missingDates = np.arange(dataDates[0],curDates[0],np.timedelta64(1,'D'))
        allDates = np.append(missingDates,curDates)



        # Make the arrays of IFR and Vacc for all possible dates
        regIFR = dfIFR[curRegion]
        regVacc = dfVaccOnce[curRegion]
        allVacc = np.append(np.zeros((len(missingDates),)),regVacc) # Zeros until first date
        allIFR = np.append(regIFR[0]*np.ones((len(missingDates),)),regIFR) # Repeat first value until first date

        # deathsToUse = curdfData.new_deaths.values[:-1]
        deathsToUse = curdfData.new_deaths.values
        # curCase = deathsToUse/(np.divide(allIFR,100))
        curCaseMean = deathsToUse/(allIFR[0]/100)


        # DatesToPlot = allDates - np.timedelta64(21,'D')
        DatesToPlot = dataDates - np.timedelta64(21,'D')


        # ax1.plot(curdfData.date,curdfData.new_cases,'k*:',markersize=2,linewidth=0.5)
        ax1.plot(rnTime(curdfData.date,meanWidth),rnMean(curdfData.new_cases,meanWidth),'k',label='Confirmed cases')
        # ax1.plot(DatesToPlot,curCaseMean,'g*:',markersize=2,linewidth=0.5)
        ax1.plot(rnTime(DatesToPlot,meanWidth),rnMean(curCaseMean,meanWidth),'g:',label='From constant IFR')
        # # ax1.plot(DatesToPlot,curCase,'b*:',markersize=2,linewidth=0.5)
        # ax1.plot(rnTime(DatesToPlot,meanWidth),rnMean(curCase,meanWidth),'b--',label='From IFR as a function of vaccination')

        try:
            deathsToUse = curdfData.new_deaths.values[:-1]
            curCase = deathsToUse/(np.divide(allIFR,100))
            ax1.plot(rnTime(DatesToPlot,meanWidth),rnMean(curCase,meanWidth),'b--',label='From IFR as a function of vaccination')
        except:
            print('Could not do curCase')


        ax2.plot(allDates,allIFR,'b--')
        ax2.plot(allDates,np.ones(allIFR.shape)*allIFR[0],'g:')

        ax2_2 = ax2.twinx()
        clrTwin = 'm'
        ax2_2.plot(allDates,allVacc,color=clrTwin)
        ax2_2.set_ylabel('Vaccination rate [%]',color= clrTwin )
        ax2_2.tick_params(axis='y', labelcolor=clrTwin)

        
        # ax2.plot(dfIFR.Dates,regIFR,'b--')
        # ax2.plot(dfIFR.Dates,np.ones(regIFR.shape)*regIFR[0],'g:')

        # ax2_2.plot(dfVaccOnce.Dates,regVacc,color=clrTwin)

        # # Draw some straight lines on bottom plot
        # firstDateData = curdfData.date.iloc[0]
        # firstDateVacc = dfVaccOnce.Dates.iloc[0]
        # # ax2.plot([firstDateData,firstDateVacc],[regIFR[0],regIFR[0]],'b--')
        # # ax2.plot([firstDateData,firstDateVacc],[regIFR[0],regIFR[0]],'g:')
        # # ax2.plot([firstDateData,firstDateVacc],[0,0],color=clrTwin)


        ax1.set_ylim(bottom=0)
        ax2.set_ylim(bottom=0)
        ax2_2.set_ylim(bottom=0,top=100)


        ax1.legend(fontsize=16)

        ax2.set_ylabel('Effective IFR [%]')
        ax1.set_ylabel('Cases')

        ax1.xaxis.set_major_formatter(mdates.DateFormatter('%b\n%Y'))
        ax1.set_title(curRegion)

        ax1.set_xlim(left=allDates[0],right=allDates[-1])

        plt.tight_layout()

        if saveFigures:
            plt.savefig('figs/CasesFromIFRVacc/'+curRegion)
            
        ax1.set_xlim(left=np.datetime64('2021-01-01'))
        ax1.set_xlim(right=np.datetime64('2021-07-01'))
        ax1.set_ylim(top=2000)

        if saveFigures:
            plt.savefig('figs/CasesFromIFRVaccZoom/'+curRegion)


        plt.close()
    except:
        print(curRegion)